Same as local11, trying with pagerank instead of connected components
Assuming there is some problem with connected components as noted in here
https://stackoverflow.com/questions/40266712/spark-graphx-scaling-connected-components
etc


Running with local[1] 

Running time ??? days

local11 error is due to either, and is uncaught:
- ???

// Shuffle blocks(?) consumed 688 GB before crashing on `HC_client` connected components (batch_size=10000, partition_size=7086)

Using `cd  /data/tmp/tmp-docker` and `du -h`

// See `local11_neo4j_connection_count.log` Using `cpu_mem_log_plot.ipynb`




In [ ]:
%showtypes on

In [ ]:
import org.neo4j.spark._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{SQLContext, DataFrame, Row, SaveMode}
import org.apache.spark.graphx._
import org.apache.spark.graphx.lib._
import org.apache.spark.{SparkConf, SparkContext}
import org.apache.spark.rdd.{RDD}
import scala.util.control.NonFatal

# Setting spark resources
- https://stackoverflow.com/questions/29940711/apache-spark-setting-executor-instances-does-not-change-the-executors
- https://stackoverflow.com/questions/32621990/what-are-workers-executors-cores-in-spark-standalone-cluster
- https://medium.com/@thejasbabu/spark-under-the-hood-partition-d386aaaa26b7
- https://stackoverflow.com/questions/24622108/apache-spark-the-number-of-cores-vs-the-number-of-executors



### executors
    kg@bigdata-009>
    CPU(s):       64
    Mem:          252G
    
     <name>yarn.nodemanager.resource.memory-mb</name>
     <value>64960</value>

     <name>yarn.scheduler.maximum-allocation-vcores</name>
     <value>360</value>
     
     <name>yarn.nodemanager.resource.cpu-vcores</name>
     <value>36</value>
     
- You can assign the number of cores per executor with `--executor-cores`
- `yarn.nodemanager.resource.memory-mb` needs to be larger than `--executor-memory`
- Setting `spark.executor.*` only applied to `yarn` and not local
- seems better to have 3 or 4 executors per node, instead of 1 executors per node
- on each node leave ~10% memory and cpu for yarn and other overheads
- TODO: unsolved whether spark.executor.cores refers to core or vcore, can test by setting spark.executor.cores to be large number, eg 340, and if executor still can be allocated then it refers to vcore 

- yarn.scheduler.maximum-allocation-vcores controls the maximum vcores that any submitted job can request. yarn.nodemanager.resource.cpu-vcores, on the other hand, controls how many vcores can be scheduled on a particular NodeManager instance. 

### partitions
- The recommend number of partitions is around 3 or 4 times the number of CPUs in the cluster

In [ ]:
// sc.stop() // see kernel.json, see "__TOREE_OPTS__": "--nosparkcontext"
// spark.driver.memory=45g. Is set in docker/conf/spark-defaults.conf
val conf = new SparkConf().set("spark.app.name","local11").set("spark.master","local[1]").set("spark.executor.instances","1").set("spark.executor.cores","1").set("spark.executor.memory","45g")

val sc = new SparkContext(conf)
val sqlContext = new org.apache.spark.sql.SQLContext(sc)
import sqlContext.implicits._

// val neo = Neo4j(sc)

println("Spark web UI: " + "http://10.100.34.19:4040/ (or 4041...)") //spark web UI
sc.getConf.getAll.foreach(println)

In [ ]:
def pickleThis[A](sc:SparkContext, dataToSave:A, schemaString:String, savePath:String) : String = {
    val dataToSaveReady = dataToSave match {
      case dataToSave:RDD[Row] => dataToSave // val qq : RDD[Row] = loaded_df.rdd // the reverse transformation
      case dataToSave:Array[Row] => sc.parallelize(dataToSave.toList) // val qq = loaded_df.collect // the reverse transformation
// DOESNT WORK because it intercepts List[Array[Any]]
// case dataToSave:List[Array[String]] => sc.parallelize(dataToSave.map(p => Row(p: _*))) 
      case dataToSave:List[Array[_]] => {
          val tmp = dataToSave.map(row => row.map(el => el.toString))
          sc.parallelize(tmp.map(p => Row(p: _*))) 
      }
      case _ => throw new Exception("Wrong argument type")
    }

  val schema =
    StructType(
      schemaString.split(" ").map(fieldName => StructField(fieldName, StringType, true)))

  val dfToSave = sqlContext.createDataFrame(dataToSaveReady, schema)

  dfToSave.write.mode("overwrite").save(savePath)
  println("data overwritten")
    
  return s"""val loaded_df = sqlContext.read.load(\"$savePath\")"""
}

In [ ]:
// // val query = "MATCH p=()-[r:Person_relation]->() RETURN count(r)"
// // val query = "MATCH (n:Person) RETURN count(n)"

// val query = s"""MATCH (a) WITH DISTINCT LABELS(a) AS temp, COUNT(a) AS tempCnt
// UNWIND temp AS label
// RETURN label, SUM(tempCnt) AS cnt"""

// val cursor = Executor.execute(sc, query, Map(("","")))
// val response = cursor.rows.toList

// val label_counts = response.sortWith(_(1).asInstanceOf[Long] > _(1).asInstanceOf[Long])
// pickleThis(sc, label_counts, "label count_n", "file:///home/jovyan/work/hadoop-client-scala-prod/pickles/label_counts.pickle")

In [ ]:
// val label_counts_df = sqlContext.read.load("file:///home/jovyan/work/hadoop-client-scala-prod/pickles/label_counts.pickle")
// label_counts_df.show

In [ ]:
// val label_rel_counts = label_counts.map( row => {
//     val str = row(0)
//     val query = s"MATCH (n:$str)-[r]-() return COUNT(r)"
//     val cursor = Executor.execute(sc, query, Map(("","")))
//     val response = cursor.rows.toList
//     row :+ response(0)(0)
// })
// pickleThis(sc, label_rel_counts, "label count_n count_r", "file:///home/jovyan/work/hadoop-client-scala-prod/pickles/label_rel_counts.pickle")

In [ ]:
var label_rel_counts_df = sqlContext.read.load("file:///home/jovyan/work/hadoop-client-scala-prod/pickles/label_rel_counts.pickle")

### casting columns to the right types, since we didnt save it right

In [ ]:
label_rel_counts_df = label_rel_counts_df.withColumn("count_n", label_rel_counts_df("count_n").cast("int"))
label_rel_counts_df = label_rel_counts_df.withColumn("count_r", label_rel_counts_df("count_r").cast("int"))
label_rel_counts_df.printSchema
label_rel_counts_df.show

In [ ]:
// val neo = Neo4j(sc)
// var neo4j : Neo4j = null.asInstanceOf[Neo4j]
// var graph : Graph[Long, String] = null.asInstanceOf[Graph[Long, String]]
// var graphSaveThis : Graph[VertexId,String] = null.asInstanceOf[Graph[VertexId, String]]

// neo4j = neo.rels(s"MATCH (n:Address)-[r]-(m) RETURN id(n) as src, id(m) as dst, type(r) as value SKIP {_skip} LIMIT {_limit}").partitions(10).batch(20)
// graph = neo4j.loadGraph[Long,String]

In [ ]:
// class Neo4jExperiment
import org.neo4j.spark.{Neo4j, Neo4jGraph}
import org.apache.spark.graphx.{Graph, VertexId}
import org.apache.spark.graphx.lib.{PageRank}


class Neo4jExperiment(val sc:SparkContext, val partitions:Int=12, val batch:Int=125) {
    val neo = Neo4j(sc)
    var neo4j : Neo4j = null.asInstanceOf[Neo4j]
    var graph : Graph[Long, String] = null.asInstanceOf[Graph[Long, String]]
//     var graphSaveThis : Graph[VertexId,String] = null.asInstanceOf[Graph[VertexId, String]]
    var graphSaveThis : Graph[Double,Double] = null.asInstanceOf[Graph[Double, Double]]
    
    def get_count_load_time(label:String) : Double = {
        println("label: " ++ label)
        println("partitions: " ++ partitions.toString)
        println("batch: " ++ batch.toString)
        
        val t_start = System.currentTimeMillis()
        try {
            neo4j = neo.rels(s"MATCH (n:$label)-[r]-(m) RETURN id(n) as src, id(m) as dst, type(r) as value SKIP {_skip} LIMIT {_limit}").partitions(partitions).batch(batch)
            graph = neo4j.loadGraph[Long,String]
            println("get_count_load_time ending")
            (System.currentTimeMillis() - t_start)/1000d
        } catch {
          case NonFatal(e) => {
              println("==" * 20 ++ "get_count_load_time_SDERRORSTART")
              println("==" * 20)
              e.printStackTrace
              println("==" * 20 ++ "get_count_load_time_SDERROREND")
              println("==" * 20)
              (System.currentTimeMillis() - t_start)/1000d + 1e8 // 1e8 seconds = 3 years
          }
        }

    }
    
    def get_count_n_time(label:String) : Double = {
        val t_start = System.currentTimeMillis()
        try {
            println(graph.vertices.count)
            (System.currentTimeMillis() - t_start)/1000d
        } catch {
          case NonFatal(e) => {
              println("==" * 20 ++ "get_count_n_time_SDERRORSTART")
              println("==" * 20)
              e.printStackTrace
              println("==" * 20 ++ "get_count_n_time_SDERROREND")
              println("==" * 20)
              (System.currentTimeMillis() - t_start)/1000d + 1e8 // 1e8 seconds = 3 years
          }
        }
    }

    def get_count_r_time(label:String) : Double = {        
        val t_start = System.currentTimeMillis()
        try {
            println(graph.edges.count)
            (System.currentTimeMillis() - t_start)/1000d
        } catch {
          case NonFatal(e) => {
              println("==" * 20 ++ "get_count_r_time_SDERRORSTART")
              println("==" * 20)
              e.printStackTrace
              println("==" * 20 ++ "get_count_r_time_SDERROREND")
              println("==" * 20)
              (System.currentTimeMillis() - t_start)/1000d + 1e8 // 1e8 seconds = 3 years
          }
        }
    }

    def get_count_cc_time(label:String) : Double = {
        val t_start = System.currentTimeMillis()
        try {
            // Find pagerank, 5 iterations
            graphSaveThis = PageRank.run(graph,5)
            println("get_count_cc_time ending")
            (System.currentTimeMillis() - t_start)/1000d
        } catch {
          case NonFatal(e) => {
              println("==" * 20 ++ "get_count_cc_time_SDERRORSTART")
              println("==" * 20)
              e.printStackTrace
              println("==" * 20 ++ "get_count_cc_time_SDERROREND")
              println("==" * 20)
              (System.currentTimeMillis() - t_start)/1000d + 1e8 // 1e8 seconds = 3 years
          }
        }
    }

    def get_count_save_time(label:String) : Double = {
        val t_start = System.currentTimeMillis()
        try {
            val return_val = Neo4jGraph.saveGraph(sc, graphSaveThis, "sdComponent")
            println("saveGraph: " ++ return_val.toString) // how many nodes has been written
            (System.currentTimeMillis() - t_start)/1000d
        } catch {
          case NonFatal(e) => {
              println("==" * 20 ++ "get_count_save_time_SDERRORSTART")
              println("==" * 20)
              e.printStackTrace
              println("==" * 20 ++ "get_count_save_time_SDERROREND")
              println("==" * 20)
              (System.currentTimeMillis() - t_start)/1000d + 1e8 // 1e8 seconds = 3 years
          }
        }
    }
}

# Start experiment

In [ ]:
// select a batch_size small enough for spark.executor.memory=45g, apparently 125000 was too big
var df = label_rel_counts_df.filter($"count_r" > 0).sort($"count_n")   // $"count_n".desc
val batch_size = 10000
df = df.withColumn("batch_size", lit(batch_size) ).withColumn("partition_size", ceil($"count_r"/batch_size) )
df.show

In [ ]:
var df = label_rel_counts_df.filter($"count_r" > 0).sort($"count_n")   // $"count_n".desc
// val batch_size = 10000 // defined in cell above
df = df.withColumn("batch_size", lit(batch_size) ).withColumn("partition_size", ceil($"count_r"/batch_size) )

var count_load_time:Seq[(String, Double)] = Seq.empty
var count_n_time:Seq[(String, Double)] = Seq.empty
var count_r_time:Seq[(String, Double)] = Seq.empty
var count_cc_time:Seq[(String, Double)] = Seq.empty
var count_save_time:Seq[(String, Double)] = Seq.empty

df.collect.foreach(row => {
    val label = row(0).toString
    val partition_size:Int = df.filter($"label" === s"$label").select("partition_size").head.get(0).toString.toInt
    val batch_size:Int = df.filter($"label" === s"$label").select("batch_size").head.get(0).toString.toInt
    
    val neo4jExperiment = new Neo4jExperiment(sc, partition_size, batch_size) // new Neo4jExperiment(sc, 12, 125) // new Neo4jExperiment(sc, 1200, 125000)
    count_load_time = count_load_time :+ (label, neo4jExperiment.get_count_load_time(label))
    count_n_time = count_n_time :+ (label, neo4jExperiment.get_count_n_time(label))
    count_r_time = count_r_time :+ (label, neo4jExperiment.get_count_r_time(label))
    count_cc_time = count_cc_time :+ (label, neo4jExperiment.get_count_cc_time(label))
    count_save_time = count_save_time :+ (label, neo4jExperiment.get_count_save_time(label))
})

In [ ]:
// putting results in the df
val dftemp = count_load_time.toDF("label","count_load_time")
df = df.join(dftemp, Seq("label"), "left")

val dftemp = count_n_time.toDF("label","count_n_time")
df = df.join(dftemp, Seq("label"), "left")

val dftemp = count_r_time.toDF("label","count_r_time")
df = df.join(dftemp, Seq("label"), "left")

val dftemp = count_cc_time.toDF("label","count_cc_time")
df = df.join(dftemp, Seq("label"), "left")

val dftemp = count_save_time.toDF("label","count_save_time")
df = df.join(dftemp, Seq("label"), "left")

df.show

# Experiment results

In [ ]:
df.write.mode("overwrite").save("file:///home/jovyan/work/hadoop-client-scala-prod/pickles/local09_sdout.pickle")

In [ ]:
val local08_sdout = sqlContext.read.load("file:///home/jovyan/work/hadoop-client-scala-prod/pickles/local09_sdout.pickle")
local08_sdout.printSchema
local08_sdout.show

In [ ]:
// // pattern is using `id(n)` if node property is set to null, like example below
// val t_start = System.currentTimeMillis()

// // val g = Neo4jGraph.loadGraph(sc, "Person", Seq("Person_relation"), "Person")
// val graph = neo.pattern(("Person",null),("Person_relation",null),("Person",null)).partitions(40).batch(850000).loadGraph[Long,Long]

// val t_end = System.currentTimeMillis()
// println("Elapsed time: " + (t_end - t_start)/1000d + "s")

// g.vertices.take(5)

// val udfSquared = udf((num: Int) => {
//     num - 9999
//   }
// )
// df.withColumn("count_n2", udfSquared($"count_n")).show

In [ ]:
// // Pickling example
// label_counts.foreach(row => println(row.deep.mkString("\t\t")))
// val label_counts_casted:List[(String,String)] = label_counts.map(el => (el(0).toString,el(1).toString))
// val label_counts_df = sc.parallelize(label_counts_casted,1).toDS().toDF()
// label_counts_df.write.mode("overwrite").save("file:///home/jovyan/work/hadoop-client-scala-prod/pickles/label_counts_df.pickle")
// // val label_counts_df = sqlContext.read.load("file:///home/jovyan/work/hadoop-client-scala-prod/pickles/label_counts_df.pickle")
// label_counts_df.show
// label_counts_df.collect.foreach(el => println(el(0), el(1)))

In [ ]:
// turning df to Array[Seq[String]] with null values
// df.collect.map(row => row.toSeq.map(el => Option(el).getOrElse("null").toString))